## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.

In [74]:
from math import prod
import numpy as np

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def f_j(w):
    return (np.arange(1, w.shape[-1]+1) * 200)/ (2 * w.shape[-1])

# funções de extração de características

# Dominio do tempo
def wamp(x, threshold = 0):
    return np.sum(np.abs(np.diff(x)) > threshold, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def wl(x):
    return np.abs(np.diff(x, axis=-1)).sum(axis=-1)

def zc(x):
    return np.sum(np.diff(np.sign(x), axis=-1) != 0, axis=-1)

# Dominio da frequencia
def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def fmn(w):
    return np.sum(f_j(w) * PSD(w)) / fmd(w)*2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def mmnf(w):
    return np.sum(f_j(w) * np.abs(w), axis=-1) / mmdf(w)*2


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor

In [75]:
# carregando dados do "prepare.ipynb"

x = np.load("dataset/segmented_data_t.npy")
w = np.load("dataset/segmented_data_f.npy")

print("Shape dos vetores orginais:", x.shape, w.shape)

# aplicando características
features = np.array([
  wamp(x),
  var(x),
  rms(x),
  wl(x),
  zc(x),
  
  fmd(w),
  fmn(w),
  mmdf(w),
  mmnf(w)
])

print(features.shape, "(caracts, participantes, ensaios, eletrodos, janelas)")

Shape dos vetores orginais: (2, 28, 2, 20, 64) (2, 28, 2, 20, 33)
(9, 2, 28, 2, 20) (caracts, participantes, ensaios, eletrodos, janelas)


In [76]:
features = features.transpose(1, 0, 3, 2, 4)
features = features.reshape(features.shape[0], features.shape[1] * features.shape[2], features.shape[3] * features.shape[4])
features = features.swapaxes(1, 2)

print(features.shape, "(participantes, instancias, caracteristicas)")

(2, 560, 18) (participantes, instancias, caracteristicas)


Criação do vetor de *labels*

In [77]:
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

# print(labels_num)

# criação do vetor de labels final
y = np.repeat(labels_num, int(features.shape[1] / len(labels_num)), axis = None)

# print(y)
len(y)

560

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [78]:
# aplicando normalização
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

for i in range(len(features)):
  features[i] = ss.fit_transform(features[i])

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

In [79]:
import warnings
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE

warnings.filterwarnings('ignore')

data = {}

for i in range(len(features)):
    
    kb = SelectKBest(f_classif, k = 10).fit_transform(features[i], y)
    data[f'KBestFeatParticipante{i}'] = kb
    # print(kb.shape)

    rfe = RFE(svm.SVC(kernel="linear"), n_features_to_select = 10).fit_transform(features[i], y)
    data[f'RFEFeatParticipante{i}'] = rfe
    # print(rfe.shape)

    model = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state = 42)).fit_transform(features[i], y)
    data[f'ModelFeatParticipante{i}'] = model
    # print(model.shape)

*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [80]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import metrics
from sklearn.svm import SVC

# aplicando a classificação
for key, X in data.items():
    print(f'----- Classificando com {key} -----')

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    param_grid = [
        {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
        {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']},
        {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['poly']}
    ]

    grid = GridSearchCV(SVC(), param_grid, cv=5, n_jobs = -1)
    grid.fit(x_train, y_train)

    y_pred = grid.predict(x_test)
    print(f'Acurácia com {key}: {metrics.accuracy_score(y_test, y_pred)}\n')

----- Classificando com KBestFeatParticipante0 -----
Acurácia com KBestFeatParticipante0: 0.4642857142857143

----- Classificando com RFEFeatParticipante0 -----
Acurácia com RFEFeatParticipante0: 0.44642857142857145

----- Classificando com ModelFeatParticipante0 -----
Acurácia com ModelFeatParticipante0: 0.5089285714285714

----- Classificando com KBestFeatParticipante1 -----
Acurácia com KBestFeatParticipante1: 0.29464285714285715

----- Classificando com RFEFeatParticipante1 -----
Acurácia com RFEFeatParticipante1: 0.3392857142857143

----- Classificando com ModelFeatParticipante1 -----
Acurácia com ModelFeatParticipante1: 0.24107142857142858

